# Probabilistic Time Series Analysis

## Week 5: Hidden Markov Models

---

You will need the following new Python package. Install it either with `conda install -c conda-forge` if you use the Anaconda environment or `pip install`.

    hmmlearn

Places where you are supposed to fill in code are marked

    #
    # TODO: some instructions
    # 
    
The rest of the code we will run and discuss if time permits, otherwise try it out at home and try to answer the questions mentioned in the text boxes for yourself.

### Please turn in the code before 10/10/2018 5:20pm. 

### Your work will be evaluated based on the code and plots. You don't need to write down your answers to other questions in the text blocks, just think them over.

### Title your submission file `lab5-student-[YOUR NET ID].ipynb`.

---

## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import random
import pylab
from collections import Counter
# from hmmlearn import hmm
import time

## Part I: Data Loading

Load the Wall Street Journal POS dataset. Transform them into indices. 

In [2]:
train_dir = "../../data/en-wsj-train.pos"
test_dir = "../../data/en-wsj-dev.pos"

def load_pos_data(data_dir, word_indexer=None, label_indexer=None, top_k=20000):
    """
    Function that loads the data
    """
    with open(data_dir, "r") as f:
        # load data
        content = f.readlines()
        intermediate_X = []
        intermediate_z = []
        current_X = []
        current_z = []
        vocab_counter = Counter()
        label_set = set()
        for line in content:
            tokens = line.replace("\n", "").replace("$", "").split("\t")
            if len(tokens) <= 1:
                intermediate_X.append(current_X)
                intermediate_z.append(current_z)
                current_X = []
                current_z = []
            elif len(tokens[1]) > 0:
                vocab_counter[tokens[0]]+=1
                label_set.add(tokens[1])
                current_X.append(tokens[0].lower())
                current_z.append(tokens[1])
        # index data
        top_k_words = vocab_counter.most_common(top_k)
        # 0 is reserved for unknown words
        word_indexer = word_indexer if word_indexer is not None else dict([(top_k_words[i][0], i+1) for i in range(len(top_k_words))])
        word_indexer["UNK"] = 0 
        label_indexer = label_indexer if label_indexer is not None else dict([(label, i) for i, label in enumerate(label_set)])
        output_X = []
        output_z = []
        current_X = []
        current_z = []
        for i in range(len(intermediate_X)):
            for j in range(len(intermediate_X[i])):
                if intermediate_X[i][j] in word_indexer:
                    current_X.append(word_indexer[intermediate_X[i][j]])
                else:
                    current_X.append(0)
                current_z.append(label_indexer[intermediate_z[i][j]])
            # populate holders
            output_X.append(current_X)
            output_z.append(current_z)
            # reset current holder
            current_X = []
            current_z = []
        return output_X, output_z, label_indexer, word_indexer, {v: k for k, v in label_indexer.items()}, {v: k for k, v in word_indexer.items()}

In [3]:
train_X, train_z, label_indexer, word_indexer, label_lookup, vocab_lookup = load_pos_data(train_dir)
test_X, test_z, _, _, _, _ = load_pos_data(test_dir, word_indexer=word_indexer, label_indexer=label_indexer)

## Part II: HMM Implementation

In this part, you will implement the Hidden Markov Model with following methods:


- sample: given a initial state and the number of steps, returns a sequence of sampled states and observations.


- fit: update the transition matrix, emission matrix, and the initial state probability. Note that in our case, the hidden states are given. We don't need to use EM for the learning. Just count the number of times that given transitions / emissions / initial states appear, and normalize those counts to fill in the parameters.


- decode_single_chain: use the Viterbi Algorithm to find the most probable sequence of states.

In [17]:
def reconstruct_sequence(idx_list, lookup):
    """
    Function that reconstructs a sequence of index
    """
    return [lookup[x] for x in idx_list]

def percentage_agree(x, y):
    """
    Function that shows the % of agreement among two list
    """
    assert len(x)==len(y)
    return float(np.sum(np.array(x)==np.array(y)))/len(x)

class MyHMM:
    def __init__(self, num_unique_states, num_observations):
        """
        Constructor
        @param num_unique_states: # of unique states (POS Tags)
        @param num_observations: # of unique observations (words)
        """
        self.EPS = 1e-100
        self.num_unique_states = num_unique_states
        self.num_observations = num_observations
        self.transition_matrix = np.zeros((num_unique_states, num_unique_states))
        self.emission_matrix = np.zeros((num_unique_states, num_observations))
        self.initial_states_vector = np.zeros(num_unique_states)
    
    def fit(self, X, z):
#         fit: update the transition matrix, emission matrix, 
# and the initial state probability. Note that in our case, 
#the hidden states are given. We don't need to use EM for the learning.
#Just count the number of times that given transitions / emissions / 
#initial states appear, and normalize those counts to fill in 
#the parameters.

        """
        Method that fits the model.
        @param X: array-like with dimension [# of examples, # of length]
        @param z: array-like with dimension [# of examples, # of length]
        """
        #
        # TODO: Add your implementation here
        #
        assert(len(X) == len(z))
        
        for sentence_id in range(len(X)):
            sentence = X[sentence_id]
            tags     = z[sentence_id]
            
            # learn initial
            self.initial_states_vector[tags[0]] += 1
            
            # learn tags and tags to word
            for i in range(len(sentence) - 1):
                self.transition_matrix[tags[i]][tags[i+1]] += 1
                self.emission_matrix[tags[i]][sentence[i]] += 1
                
            # do not forget about the last one
            self.emission_matrix[tags[-1]][sentence[-1]] += 1
                
        # normalizing
        # obviously, we need some smoothing (like Laplace smoothing or others)
        # to make it work better
        # here we use simple counter approach
        self.initial_states_vector /= self.initial_states_vector.sum()
        
        self.transition_matrix = (self.transition_matrix.T /
                                  self.transition_matrix.sum(axis=1).T).T
        self.emission_matrix = (self.emission_matrix.T / 
                                self.emission_matrix.sum(axis=1).T).T
        
        
        # for computational stability and shorter names
        self.log_z_0 = np.log(self.initial_states_vector + self.EPS)
        self.log_z_z = np.log(self.transition_matrix + self.EPS)
        self.log_z_x = np.log(self.emission_matrix + self.EPS)

        
        return None
    
    def decode_single_chain(self, sentence):
        """
        Auxiliary method that uses Viterbi on single chain
        @param X: array-like with dimension [ # of length]
        @return z: array-like with dimension [# of length]
        """     
        #
        # TODO: Add your implementation here
        #
        
        # we will use these matrices
        prob_matrix = [[-1000000] * len(sentence) 
                       for _ in range(self.num_unique_states)
                      ]
        # to recover answer
        back_matrix = [[-1] * len(sentence)  
                       for _ in range(self.num_unique_states)
                      ]

        # initialization
        for i in range(self.num_unique_states):
            prob_matrix[i][0] = (self.log_z_0[i]
                                 + self.log_z_x[i][sentence[0]])
            
        for column in range(1, len(sentence)):
            current_word = sentence[column]
            for current_tag in range(self.num_unique_states):
                for prev_tag in range(self.num_unique_states):
                    new_prob = (prob_matrix[prev_tag][column - 1]
                                + self.log_z_z[prev_tag][current_tag]
                                + self.log_z_x[current_tag][current_word])
                    if new_prob > prob_matrix[current_tag][column] and new_prob:
                        prob_matrix[current_tag][column] = new_prob
                        back_matrix[current_tag][column] = prev_tag


        result = []
        result.append(np.argmax([row[-1] 
                                for row in prob_matrix]))

        for i in range(len(sentence) - 1, 0, -1):
            result.append(back_matrix[result[-1]][i])

    #     print(tags)
    #     print(*prob_matrix, sep='\n')
    #     print(*back_matrix, sep='\n')
        
        return result[::-1]
        
    def decode(self, X):
        """
        Method that performs the Viterbi the model.
        @param X: array-like with dimension [# of examples, # of length]
        @return z: array-like with dimension [# of examples, # of length]
        """
        return [self.decode_single_chain(sample) for sample in X]
    
    def sample(self, n_step, initial_state):
        """
        Method that given initial state and produces n_step states and observations
        @param n_step: integer
        @param initial_state: an integer indicating the state
        """
        #
        # TODO: Add your implementation here
        #
        states = []  # aka z
        observations = []  # aka x
        
        cur_state = initial_state
        cur_observation = np.random.choice(self.num_observations, 
                                           p=self.emission_matrix[cur_state])
        
        states.append(cur_state)
        observations.append(cur_observation)
        
        for i in range(n_step-1):
            cur_state = np.random.choice(self.num_unique_states, 
                                         p=self.transition_matrix[cur_state])
            cur_observation = np.random.choice(self.num_observations, 
                                               p=self.emission_matrix[cur_state])
        
            states.append(cur_state)
            observations.append(cur_observation)
            
        return states, observations

In [18]:
sample_HMM = MyHMM(num_unique_states=len(label_indexer), 
                   num_observations=len(word_indexer))

In [20]:
sample_HMM.fit(train_X, train_z)

In [21]:
sample_HMM.log_z_0

array([-230.2585093 ,   -8.51255746,   -2.07061461,   -3.18711143,
         -7.81941028,   -7.22470317,   -4.40579038,   -5.96702619,
         -8.10709235, -230.2585093 ,   -5.62218571,   -2.58463194,
         -6.41761173,   -7.1580118 ,   -7.95294168,   -7.25979449,
         -7.06563848,   -5.84706688, -230.2585093 ,   -6.58466582,
         -8.64608886,   -1.60492719,   -7.8839488 , -230.2585093 ,
         -1.52818843,   -2.84095389,   -2.86666897,   -2.67409842,
         -3.20367115,   -5.13241349,   -3.17802871,   -6.53155599,
         -6.19754985,   -4.39963652,   -5.77171744,   -5.46803503,
         -6.14934775,   -7.37312318,  -10.591999  ,   -5.91917017,
       -230.2585093 ,   -5.82982507,   -5.07454611, -230.2585093 ])

In [23]:
sample = sample_HMM.sample(10, 5)

In [24]:
reconstruct_sequence(sample[0], label_lookup)

['PDT', 'DT', 'NN', 'NN', 'IN', 'CD', 'NNS', 'TO', 'VB', 'DT']

In [25]:
reconstruct_sequence(sample[1], vocab_lookup)

['all',
 'a',
 'revenue',
 'proposal',
 'between',
 '640',
 'officials',
 'to',
 'invest',
 'some']

In [26]:
sample_HMM.decode_single_chain(sample[0])

[2, 25, 43, 43, 24, 3, 14, 13, 16, 2]

## Part III: Validation

### Learn an HMM

In [27]:
num_states = len(label_indexer)
num_obs = len(word_indexer)
my_hmm = MyHMM(num_states, num_obs)
my_hmm.fit(train_X, train_z)

### Decode

In [28]:
i = 5
res = my_hmm.decode_single_chain(test_X[i])
print("data: {0} \n pred: {1} \n label: {2}".format(reconstruct_sequence(test_X[i], vocab_lookup),
                                                    reconstruct_sequence(res, label_lookup),
                                                  reconstruct_sequence(test_z[i], label_lookup)))

data: ['this', 'financing', 'system', 'was', 'created', 'in', 'the', 'new', 'law', 'in', 'order', 'to', 'keep', 'the', 'bailout', 'spending', 'from', 'swelling', 'the', 'budget', 'deficit', '.'] 
 pred: ['DT', 'NN', 'NN', 'VBD', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NN', 'TO', 'VB', 'DT', 'NN', 'NN', 'IN', 'VBG', 'DT', 'NN', 'NN', '.'] 
 label: ['DT', 'NN', 'NN', 'VBD', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NN', 'TO', 'VB', 'DT', 'NN', 'NN', 'IN', 'VBG', 'DT', 'NN', 'NN', '.']


In [29]:
start_time = time.time()
pred_train = my_hmm.decode(train_X[:1000])
print("takes {0} seconds".format(time.time() - start_time))

takes 46.78345489501953 seconds


In [30]:
start_time = time.time()
pred_test = my_hmm.decode(test_X)
print("takes {0} seconds".format(time.time() - start_time))

takes 78.61477279663086 seconds


In [31]:
np.mean([percentage_agree(pred_train[i], train_z[i]) for i in range(len(pred_train))])

0.9320476845713684

In [32]:
np.mean([percentage_agree(pred_test[i], test_z[i]) for i in range(len(pred_test))])

0.9204388549707038

### Sample

In [33]:
pos_tag, words = my_hmm.sample(10, label_indexer["NNP"])
print(reconstruct_sequence(pos_tag, label_lookup))
print(reconstruct_sequence(words, vocab_lookup))

['NNP', 'NN', ',', 'DT', 'NN', 'IN', 'CD', 'NNS', 'CC', 'DT']
['UNK', 'stock', ',', 'the', 'UNK', 'in', 'billion', 'companies', 'and', 'the']
